In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from tqdm import tqdm

from utils.preprocessing import get_texts
from utils.preprocessing import get_texts, stop_words

In [2]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")

In [3]:
df_esg_score["sector"].unique()

array(['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare',
       'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology',
       'Financial Services', 'Communication Services', 'Real Estate', nan],
      dtype=object)

In [4]:
sector = "Technology"
# sector = "Energy"
tickers = df_esg_score[df_esg_score["sector"] == sector]["Company"]
esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]

In [5]:
len(tickers)

63

In [6]:
# score_type = "governanceScore"
# score_type = "environmentScore"
score_type = "socialScore"

In [7]:
score = esgs[score_type]

In [8]:
alpha = 0.1
upper_score = np.quantile(score, 1 - alpha)
lower_score = np.quantile(score, alpha)

In [9]:
bad_companies = esgs[esgs[score_type] > upper_score]["Company"].values
good_companies = esgs[esgs[score_type] < lower_score]["Company"].values

In [10]:
good_companies

array(['QCOM', 'TRMB', 'AMAT', 'IPGP', 'STX', 'KEYS', 'CDW'], dtype=object)

In [11]:
bad_companies

array(['FTV', 'IT', 'VRSN', 'PTC', 'FTNT', 'FFIV', 'FIS'], dtype=object)

In [12]:
esgs[esgs[score_type] > upper_score]

,Company,socialScore,governanceScore,environmentScore
22,FTV,14.18,5.53,15.63
177,IT,17.12,6.28,0.11
211,VRSN,12.87,6.28,3.04
259,PTC,12.72,5.66,1.24
260,FTNT,12.13,6.17,1.23
272,FFIV,12.49,5.40,1.26
295,FIS,11.14,5.83,1.27


In [13]:
bad_companies_score = esgs[esgs[score_type] > upper_score][score_type].values
good_companies_score = esgs[esgs[score_type] < lower_score][score_type].values

In [14]:
avg_bad = np.mean(bad_companies_score)
avg_good = np.mean(good_companies_score)
print(avg_bad, avg_good)

13.235714285714284 3.315714285714286


In [15]:
print(upper_score, lower_score)

11.058000000000002 3.862


In [16]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))
good_cik = []
bad_cik = []
for ticker in good_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        good_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        good_cik.append('')

for ticker in bad_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        bad_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        bad_cik.append('')

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
ret_good = get_texts(good_cik, good_companies)

0it [00:00, ?it/s]

Scraping CIK 0000804328


100%|██████████| 1/1 [00:06<00:00,  6.34s/it]


Scraping CIK 0000804328


100%|██████████| 1/1 [00:10<00:00, 10.25s/it]
1it [00:28, 28.48s/it]

...

100%|██████████| 1/1 [00:10<00:00, 10.56s/it]
6it [02:47, 27.79s/it]

Scraping CIK 0001402057


100%|██████████| 1/1 [00:07<00:00,  7.28s/it]


Scraping CIK 0001402057


100%|██████████| 1/1 [00:09<00:00,  9.05s/it]
7it [03:13, 27.58s/it]


In [18]:
ret_bad = get_texts(bad_cik, bad_companies)

0it [00:00, ?it/s]

Scraping CIK 0001659166


100%|██████████| 1/1 [00:07<00:00,  7.23s/it]


Scraping CIK 0001659166


100%|██████████| 1/1 [00:11<00:00, 11.21s/it]
1it [00:29, 29.80s/it]

...

100%|██████████| 1/1 [00:09<00:00,  9.56s/it]
6it [02:28, 23.95s/it]

Scraping CIK 0001136893


100%|██████████| 1/1 [00:06<00:00,  6.88s/it]


Scraping CIK 0001136893


100%|██████████| 1/1 [00:09<00:00,  9.68s/it]
7it [02:52, 24.59s/it]


In [19]:
good_docs = ret_good["docs"]
bad_docs = ret_bad["docs"]

In [53]:
n_min = 2
n_max = 5
cv = CountVectorizer(max_df=0.8, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
word_count_vector = cv.fit_transform(good_docs + bad_docs)

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [54]:
feature_names = cv.get_feature_names()

In [55]:
count_feature = word_count_vector.toarray().sum(axis=0)

In [56]:
feature_names = cv.get_feature_names()

In [57]:
d = {"word": [], "good_score": [], "bad_score": [], "good_score_all": []
    , "bad_score_all": [], "count": [], "good_nums": [], "bad_nums": []}

for feature_idx, word in enumerate(feature_names):
    good_sum = bad_sum = good_num = bad_num = 0

    for i, doc_set in enumerate(good_docs):
        if word in doc_set:
            good_num += 1
            good_sum += good_companies_score[i]
    for i, doc_set in enumerate(bad_docs):
        if word in doc_set:
            bad_num += 1
            bad_sum += bad_companies_score[i]
    
    # print("word: {}".format(word))
    d["word"].append(word) 
    
    if good_num:
        d["good_score"].append(good_sum / good_num)
    else:
        d["good_score"].append(0)
    if bad_num:
        d["bad_score"].append(bad_sum / bad_num)
    else:
        d["bad_score"].append(0)

    d["good_score_all"].append(good_sum / len(good_docs))
    d["bad_score_all"].append(bad_sum / len(bad_docs))

    d["count"].append(count_feature[feature_idx])
    d["good_nums"].append(good_num)
    d["bad_nums"].append(bad_num)


In [58]:
df = pd.DataFrame(data=d)


In [59]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums
0,accompanying note,3.100000,13.235714,1.771429,13.235714,349,4,7
1,active market,3.286000,13.296667,2.347143,11.397143,178,5,6
2,additional information regarding,3.500000,13.980000,1.500000,7.988571,204,3,4
3,adverse effect result,3.281667,13.150000,2.812857,9.392857,186,6,5
4,adversely affect financial,3.286000,12.588333,2.347143,10.790000,259,5,6


In [60]:
df["diff"] = abs(df["good_nums"] - df["bad_nums"])

In [61]:
df = df.sort_values("diff", ascending=False)#.head(60)

In [62]:
df.head(20)

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums,diff
55,ended june compared,3.405000,13.235714,0.972857,13.235714,251,2,7,5
57,ended october,3.280000,0.000000,2.342857,0.000000,200,5,0,5
90,june compared,3.405000,13.235714,0.972857,13.235714,265,2,7,5
162,senior vice,3.315714,13.335000,3.315714,3.810000,183,7,2,5
111,month period,0.000000,12.632500,0.000000,7.218571,461,0,4,4
72,fiscal compared,3.150000,0.000000,1.800000,0.000000,177,4,0,4
94,march compared,3.433333,13.235714,1.471429,13.235714,180,3,7,4
137,period ended june,3.433333,13.235714,1.471429,13.235714,185,3,7,4
174,statement operation,3.248333,14.920000,2.784286,4.262857,238,6,2,4
106,month ended june,3.433333,13.235714,1.471429,13.235714,1394,3,7,4


In [66]:
goodvbad_path = os.path.join("data", "goodvbad")
if not os.path.isdir(goodvbad_path):
    os.mkdir(goodvbad_path)

In [64]:
df.to_csv("data/goodvbad/{}_{}_{}_n{}-{}.csv".format(sector[:8], score_type[:3], alpha, n_min, n_max))

## Appendix

In [ ]:
# To remove all the previous files earlier than 2020
for cik in tqdm(bad_cik):
    tenk_path = os.path.join("data", "10k", cik, "rawtext")
    all_raws = os.listdir(tenk_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenk_path, filename))
    
    tenq_path = os.path.join("data", "10q", cik, "rawtext")
    all_raws = os.listdir(tenq_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenq_path, filename))


In [4]:
df = pd.read_csv("energy_good_vs_bad_uni_bi_tri.csv", index_col=0)

In [5]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,good_nums,bad_nums,diff
0,accompanying note consolidated,9.773333,23.7600,4.886667,3.960000,3,1,2
1,accompanying note integral,10.557500,21.6950,7.038333,7.231667,4,2,2
2,acmp,0.000000,21.9800,0.000000,3.663333,0,1,1
3,activity cash flow,10.975000,21.0660,7.316667,17.555000,4,5,1
4,adjusted ebitda,0.000000,22.2825,0.000000,14.855000,0,4,4


In [237]:
comp = pd.read_csv("data/sp500_component_stocks.csv")


In [241]:
comp[comp['A'] == "C"]["Agilent Technologies Inc."]

79    Citigroup Inc.
Name: Agilent Technologies Inc., dtype: object

In [140]:

comp[comp['A'].isin(good_companies)]["Agilent Technologies Inc."]

44                   Apache Corporation
111         Cabot Oil & Gas Corporation
128                 Chevron Corporation
163                  EOG Resources Inc.
322            Marathon Oil Corporation
358    Occidental Petroleum Corporation
Name: Agilent Technologies Inc., dtype: object

In [141]:
comp[comp['A'].isin(bad_companies)]["Agilent Technologies Inc."]

215            Halliburton Company
274      Kinder Morgan Inc Class P
343    National Oilwell Varco Inc.
354                     ONEOK Inc.
411                Schlumberger NV
487        Williams Companies Inc.
Name: Agilent Technologies Inc., dtype: object